In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import math
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder
from keras import models
from keras import layers
from keras import regularizers
import matplotlib.pyplot as plt
from keras import optimizers
from numpy.random import seed
from numpy import array
from numpy import argmax
from keras import metrics
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

In [2]:
seed(10)
# read dataset
csv_file = 'assignment_ds.csv'
df = pd.read_csv(csv_file, header=None)
df = df.iloc[1: , :]
df[0:10]

,0,1,2,3,4,5,6
1,3453,0.000621,0.0773,17.08,Red,M,0
2,12098,689,7.01,0.02,Blue-white,A,3
3,2731,0.000437,0.0856,18.09,Red,M,0
4,37800,202900,6.86,-4.56,Blue,O,3
5,8570,0.00081,0.0097,14.2,Blue white,A,2
6,3605,126000,1124,-10.81,Red,M,5
7,8052,8.7,1.8,2.42,Whitish,A,3
8,3574,200000,89,-5.24,Red,M,4
9,8829,537493,1423,-10.73,White,A,5
10,3625,74000,876,-10.25,Red,M,5


In [3]:
df.shape

(200, 7)

In [4]:
# shuffle dataset
df = df.sample(frac=1)

# get inputs - outputs
inputs = df[[0,1,2,3,4,5]]
target = df [[6]]



In [17]:
# One Hot Encoding
df.groupby(4).apply(list)

4
Blue                  [0, 1, 2, 3, 4, 5, 6]
Blue White            [0, 1, 2, 3, 4, 5, 6]
Blue white            [0, 1, 2, 3, 4, 5, 6]
Blue-White            [0, 1, 2, 3, 4, 5, 6]
Blue-white            [0, 1, 2, 3, 4, 5, 6]
Orange                [0, 1, 2, 3, 4, 5, 6]
Orange-Red            [0, 1, 2, 3, 4, 5, 6]
Pale yellow orange    [0, 1, 2, 3, 4, 5, 6]
Red                   [0, 1, 2, 3, 4, 5, 6]
White                 [0, 1, 2, 3, 4, 5, 6]
White-Yellow          [0, 1, 2, 3, 4, 5, 6]
Whitish               [0, 1, 2, 3, 4, 5, 6]
Yellowish             [0, 1, 2, 3, 4, 5, 6]
Yellowish White       [0, 1, 2, 3, 4, 5, 6]
white                 [0, 1, 2, 3, 4, 5, 6]
yellow-white          [0, 1, 2, 3, 4, 5, 6]
yellowish             [0, 1, 2, 3, 4, 5, 6]
dtype: object

In [23]:
# column to lower case
df[4] = df[4].map(lambda x: x if type(x)!=str else x.lower())
df

,0,1,2,3,4,5,6
60,3575,123000,45,-6.78,red,M,4
6,3605,126000,1124,-10.81,red,M,5
21,28700,16790,6.4,-4.09,blue-white,B,3
199,39000,204000,10.6,-4.7,blue,O,3
53,3042,0.0005,0.1542,16.6,red,M,0
...,...,...,...,...,...,...,...
114,2831,0.000231,0.0915,16.21,red,M,0
65,3834,272000,1183,-9.2,red,M,5
16,17200,0.00098,0.015,12.45,blue white,B,2
126,3095,0.00019,0.492,10.87,red,M,1


In [26]:
df.groupby(4).apply(list)

4
blue                  [0, 1, 2, 3, 4, 5, 6]
blue white            [0, 1, 2, 3, 4, 5, 6]
blue-white            [0, 1, 2, 3, 4, 5, 6]
orange                [0, 1, 2, 3, 4, 5, 6]
orange-red            [0, 1, 2, 3, 4, 5, 6]
pale yellow orange    [0, 1, 2, 3, 4, 5, 6]
red                   [0, 1, 2, 3, 4, 5, 6]
white                 [0, 1, 2, 3, 4, 5, 6]
white-yellow          [0, 1, 2, 3, 4, 5, 6]
whitish               [0, 1, 2, 3, 4, 5, 6]
yellow-white          [0, 1, 2, 3, 4, 5, 6]
yellowish             [0, 1, 2, 3, 4, 5, 6]
yellowish white       [0, 1, 2, 3, 4, 5, 6]
dtype: object

In [29]:
df[4] = df[4].replace(['blue-white'], 'blue white')
df[4] = df[4].replace(['orange-red'], 'orange red')
df[4] = df[4].replace(['yellow-white', 'yellowish', 'yellowish white'], 'yellow')
df[4] = df[4].replace(['white-yellow', 'whitish'], 'white')
df

,0,1,2,3,4,5,6
60,3575,123000,45,-6.78,red,M,4
6,3605,126000,1124,-10.81,red,M,5
21,28700,16790,6.4,-4.09,blue white,B,3
199,39000,204000,10.6,-4.7,blue,O,3
53,3042,0.0005,0.1542,16.6,red,M,0
...,...,...,...,...,...,...,...
114,2831,0.000231,0.0915,16.21,red,M,0
65,3834,272000,1183,-9.2,red,M,5
16,17200,0.00098,0.015,12.45,blue white,B,2
126,3095,0.00019,0.492,10.87,red,M,1


In [30]:
df.groupby(4).apply(list)

4
blue                  [0, 1, 2, 3, 4, 5, 6]
blue white            [0, 1, 2, 3, 4, 5, 6]
orange                [0, 1, 2, 3, 4, 5, 6]
orange red            [0, 1, 2, 3, 4, 5, 6]
pale yellow orange    [0, 1, 2, 3, 4, 5, 6]
red                   [0, 1, 2, 3, 4, 5, 6]
white                 [0, 1, 2, 3, 4, 5, 6]
yellow                [0, 1, 2, 3, 4, 5, 6]
dtype: object

In [5]:
x = inputs.values #returns a numpy array
min_max_scaler = MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
inputs = pd.DataFrame(x_scaled)
inputs[0:10]

ValueError: could not convert string to float: 'Red'